# **Setup:**

In [ ]:
import glob
from PIL import Image , ImageOps
import numpy as np
import pandas as pd
import os
import random
import shutil
import yaml
import gc
from time import sleep
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# change working directory and verify files are accessible
path = '/content/drive/MyDrive/Colab_Notebooks/Final_Project'
%cd{path}
%ls -l

/content/drive/MyDrive/Colab_Notebooks/Final_Project
total 283
-rw------- 1 root root 253292 Jun  1 22:47 DataSet_Augmentations.ipynb
drwx------ 2 root root   4096 Mar 24 08:01 datasets/
drwx------ 2 root root   4096 Jun  2 05:14 Final_Product/
-rw------- 1 root root   2754 Feb 23 16:12 FinalProject_TrainingCustomDataSet.ipynb
drwx------ 2 root root   4096 Mar 11 10:01 MiniDataSet_HighRes.v9-allthedata.yolov5pytorch/
drwx------ 2 root root   4096 Jun  2 02:23 mmdetection/
-rw------- 1 root root  11978 Jun  2 06:03 SOD_HD.ipynb
drwx------ 2 root root   4096 Jun  2 02:53 Yolo-to-COCO-format-converter/


# **Donwload Dataset From Roboflow:**

In [ ]:
# Roboflow imports and installations
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = os.path.join(path, 'datasets')

In [ ]:
# download and unzip the dataset
rf = Roboflow(api_key="dEmfq24plwWwl8wWoAKj")
project = rf.workspace("hadid-excavations").project("review-project")
dataset = project.version(3).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/drive/MyDrive/Colab_Notebooks/Final_Project/datasets/Review-Project-3 in yolov5pytorch:: 100%|██████████| 880/880 [03:36<00:00,  4.07it/s]


# ***Utils For Augmentations:***

In [ ]:
def Convert_Grayscale(input_path: str , output_grayscale_path: str):

  # create the output folders
  if not os.path.exists(output_grayscale_path + 'images/'):
    os.makedirs(output_grayscale_path + 'images/')
  if not os.path.exists(output_grayscale_path + 'labels/'):
    os.makedirs(output_grayscale_path + 'labels/')

  image_names = glob.glob(input_path + 'images/*.jpg')
  for imname in image_names:  # run over all the images
    image_file_name = imname.split('/')[-1]  # find the image actual name
    image_format = imname.split('.')[-1]  # find the image format
    label_file_name = image_file_name.replace('.' + image_format,
                                              '.txt')  # change the format of the file to the labels file format
    labname = input_path + 'labels/' + label_file_name  # the corresponding label path to the current image

    image = Image.open(imname)  # open image
    gray_image = ImageOps.grayscale(image) # convert image to grayscale
    
    # create output paths for the new image and the corresponding label
    new_image_path = output_grayscale_path + 'images/' + image_file_name
    new_label_path = output_grayscale_path + 'labels/' + label_file_name

    gray_image.save(new_image_path)  # save the grayscale image
    shutil.copy2(labname , new_label_path) # save the labels list to a new destination

    image.close()
    gray_image.close()

In [ ]:
def Find_New_Class_Index(index: int , old_classes_indexes: list, new_classes_indexes: list):
  index_exist = False
  new_index = -1

  for count, i in enumerate(old_classes_indexes):
    if i == index:
      index_exist = True
      new_index = new_classes_indexes[count]
      break;

  return new_index , index_exist

In [ ]:
def Filter_Classes(input_path: str , output_filtered_path: str , old_classes_indexes: list, new_classes_indexes: list):

  # create the output folders
  if not os.path.exists(os.path.join(output_filtered_path , 'images')):
    os.makedirs(os.path.join(output_filtered_path , 'images'))
  if not os.path.exists(os.path.join(output_filtered_path , 'labels')):
    os.makedirs(os.path.join(output_filtered_path , 'labels'))

  
  image_names = glob.glob(os.path.join(input_path , 'images/*.jpg'))
  for imname in image_names:  # run over all the images
    image_file_name = imname.split('/')[-1]  # find the image actual name
    image_format = imname.split('.')[-1]  # find the image format
    label_file_name = image_file_name.replace('.' + image_format,
                                              '.txt')  # change the format of the file to the labels file format
    labname = os.path.join(input_path ,'labels' ,label_file_name)  # the corresponding label path to the current image

    # Opens:
    image = Image.open(imname)  # open image
    labels_list = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])  # read labels

    if labels_list.shape[0] > 0:
      new_labels_list = []
      for row in labels_list.iterrows(): # iretating over each label in the file (we remember that the sizes are normilized)
        new_index , index_exist = Find_New_Class_Index(int(row[1]['class']) , old_classes_indexes , new_classes_indexes)
        if index_exist:
          new_labels_list.append([new_index, row[1]['x1'], row[1]['y1'], row[1]['w'], row[1]['h']])
      
      if len(new_labels_list) > 0:
        # create the image's new path
        new_image_path = os.path.join(output_filtered_path ,'images' ,image_file_name)
        # modify the labels
        new_label_path = os.path.join(output_filtered_path ,'labels' ,label_file_name)

        image.save(new_image_path)  # save the window
        new_labels_list_df = pd.DataFrame(new_labels_list, columns=['class', 'x1', 'y1', 'w', 'h']) # create a data frame from the new labels
        new_labels_list_df.to_csv(new_label_path, sep=' ', index=False, header=False , float_format='%.6f') # save the new data frame as the new label file

        del new_labels_list_df
        gc.collect()

    # Releases:
    image.close()
    del labels_list
    gc.collect()

In [ ]:
def Update_Classes(input_path: str , data_yaml_file):
  
  # update the train and val paths in the yaml file:
  for key in data_yaml_file.keys():
    if key == 'train':
      data_file[key] = input_path + 'train/images'

    if key == 'val':
      data_file[key] = input_path + 'valid/images'

  class_index_list = []
  paths_list = []

  if os.path.exists(input_path + 'train/'):
    paths_list.append(input_path + 'train/')

  if os.path.exists(input_path + 'valid/'):
    paths_list.append(input_path + 'valid/')

  if os.path.exists(input_path + 'test/'):
    paths_list.append(input_path + 'test/')

  # find all the class indexes that left in the dataset
  for path in paths_list:
    label_names = glob.glob(path + 'labels/*.txt')
    for labname in label_names:  # run over all the labels
      labels_list = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])  # read labels
      for row in labels_list.iterrows():  # iterating over each label in the file
        if int(row[1]['class']) not in class_index_list:
          class_index_list.append(int(row[1]['class']))
      del labels_list
      gc.collect()

  # update the 'nc' field in the yaml file
  num_of_classes = len(class_index_list)
  data_yaml_file['nc'] = num_of_classes 

  # create a dictionary where the classes' names are the keys and their new indexes are the values
  total_class_names_list = data_yaml_file['names']
  class_names_dict = {}
  for count , i in enumerate(class_index_list):
    class_names_dict[total_class_names_list[i]] = count

  
  # create the list of classes names
  names_list = []
  for i in range(len(class_names_dict.keys())):
    names_list.append(None)
  
  for key in class_names_dict.keys():
    names_list[class_names_dict[key]] = key

  # update the 'names' field in the yaml file
  data_yaml_file['names'] = names_list

  # update the labels file with the new classes' indexes
  for path in paths_list:
    label_names = glob.glob(path + 'labels/*.txt')
    for labname in label_names:  # run over all the labels
      labels_list = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])  # read labels
      updated_labels = []
      for row in labels_list.iterrows():  # iterating over each label in the file
        new_class = class_names_dict[total_class_names_list[int(row[1]['class'])]]
        new_x1 = row[1]['x1'] 
        new_y1 = row[1]['y1'] 
        new_w = row[1]['w'] 
        new_h = row[1]['h'] 
        updated_labels.append([int(new_class), new_x1, new_y1, new_w, new_h])
      
      # remove existing label file
      os.remove(labname)

      # save the new label file with the same name as the removed one
      updated_labels_df = pd.DataFrame(updated_labels, 
                               columns=['class', 'x1', 'y1', 'w', 'h'])  # create a data frame from the new labels
      updated_labels_df.to_csv(labname, sep=' ', index=False, header=False, float_format='%.6f')  # save the new data frame as the new label file
      del labels_list
      gc.collect()


  return data_yaml_file 

In [ ]:
def Label_Modify_Vertical_Flip_Image(labels_list: str , output_path: str):
  labels = pd.read_csv(labels_list, sep=' ', names=['class', 'x1', 'y1', 'w', 'h']) # DataFrame of all labels in this image
  augmented_labels = []
  for row in labels.iterrows(): # iretating over each label in the file (we remember that the sizes are normilized)
    new_x1 = row[1]['x1']
    new_y1 = 1 - row[1]['y1']
    new_w = row[1]['w']
    new_h = row[1]['h']
    augmented_labels.append([int(row[1]['class']), new_x1, new_y1, new_w, new_h])

  augmented_df = pd.DataFrame(augmented_labels, columns=['class', 'x1', 'y1', 'w', 'h']) # create a data frame from the new labels
  augmented_df.to_csv(output_path, sep=' ', index=False, header=False , float_format='%.6f') # save the new data frame as the new label file


In [ ]:
def Label_Modify_Horizontal_Flip_Image(labels_list: str , output_path: str):
  labels = pd.read_csv(labels_list, sep=' ', names=['class', 'x1', 'y1', 'w', 'h']) # DataFrame of all labels in this image
  augmented_labels = []
  for row in labels.iterrows(): # iretating over each label in the file (we remember that the sizes are normilized)
    new_x1 = 1 - row[1]['x1']
    new_y1 = row[1]['y1']
    new_w = row[1]['w']
    new_h = row[1]['h']
    augmented_labels.append([int(row[1]['class']), new_x1, new_y1, new_w, new_h])

  augmented_df = pd.DataFrame(augmented_labels, columns=['class', 'x1', 'y1', 'w', 'h']) # create a data frame from the new labels
  augmented_df.to_csv(output_path, sep=' ', index=False, header=False , float_format='%.6f') # save the new data frame as the new label file

In [ ]:
def Label_Modify_Rotate90_Clockwise_Image(labels_list: str , output_path: str , height: int , width: int):
  labels = pd.read_csv(labels_list, sep=' ', names=['class', 'x1', 'y1', 'w', 'h']) # DataFrame of all labels in this image
  augmented_labels = []
  for row in labels.iterrows(): # iretating over each label in the file (we remember that the sizes are normilized)
    new_x1 = 1 - row[1]['y1']
    new_y1 = row[1]['x1']
    new_w = row[1]['h']
    new_h = row[1]['w']
    augmented_labels.append([int(row[1]['class']), new_x1, new_y1, new_w, new_h])

  augmented_df = pd.DataFrame(augmented_labels, columns=['class', 'x1', 'y1', 'w', 'h']) # create a data frame from the new labels
  augmented_df.to_csv(output_path, sep=' ', index=False, header=False , float_format='%.6f') # save the new data frame as the new label file

In [ ]:
def Label_Modify_Rotate90_ConterClockwise_Image(labels_list: str , output_path: str , height: int , width: int):
  labels = pd.read_csv(labels_list, sep=' ', names=['class', 'x1', 'y1', 'w', 'h']) # DataFrame of all labels in this image
  augmented_labels = []
  for row in labels.iterrows(): # iretating over each label in the file (we remember that the sizes are normilized)
    new_x1 = row[1]['y1']
    new_y1 = 1 - row[1]['x1']
    new_w = row[1]['h']
    new_h = row[1]['w']
    augmented_labels.append([int(row[1]['class']), new_x1, new_y1, new_w, new_h])

  augmented_df = pd.DataFrame(augmented_labels, columns=['class', 'x1', 'y1', 'w', 'h']) # create a data frame from the new labels
  augmented_df.to_csv(output_path, sep=' ', index=False, header=False , float_format='%.6f') # save the new data frame as the new label file

In [ ]:
# implement various augmentations on the images
def Image_Augmentations(image: str , labels_list: str, output_folder_path: str , horz_flip = False , vert_flip = False , rotate90clockwise = False , rotate90counterclockwise = False):

  # open the original image
  original_img = Image.open(image)
  width, height = original_img.size
  original_img.save(output_folder_path + 'images/' + image.split('/')[-1]) # save the original image to the new folder

  # save the original label file
  original_label = pd.read_csv(labels_list, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])
  original_label.to_csv(output_folder_path + 'labels/' + labels_list.split('/')[-1] , sep=' ', index=False, header=False, float_format='%.6f')

  ############################################################################
  # Flip the original image vertically
  if vert_flip:

    # flip the image
    vertical_img = original_img.transpose(method=Image.FLIP_TOP_BOTTOM)

    # save the image in the new path
    image_format = image.split('.')[-1]
    image_name = image.split('/')[-1]
    new_image_path = output_folder_path + 'images/' + image_name.replace('.' + image_format , '_vertical.' + image_format)
    vertical_img.save(new_image_path)

    # modify the labels
    label_file_format = labels_list.split('.')[-1]
    label_file_name = labels_list.split('/')[-1]
    new_label_path = output_folder_path + 'labels/' + label_file_name.replace('.' + label_file_format , '_vertical.' + label_file_format)
    Label_Modify_Vertical_Flip_Image(labels_list , new_label_path)

    vertical_img.close()


  ############################################################################
  # Flip the original image horizontally
  if horz_flip:

    # flip the image
    horz_img = original_img.transpose(method=Image.FLIP_LEFT_RIGHT)

    # save the image in the new path
    image_format = image.split('.')[-1]
    image_name = image.split('/')[-1]
    new_image_path = output_folder_path + 'images/' + image_name.replace('.' + image_format , '_horizontal.' + image_format)
    horz_img.save(new_image_path)

    # modify the labels
    label_file_format = labels_list.split('.')[-1]
    label_file_name = labels_list.split('/')[-1]
    new_label_path = output_folder_path + 'labels/' + label_file_name.replace('.' + label_file_format , '_horizontal.' + label_file_format)
    Label_Modify_Horizontal_Flip_Image(labels_list , new_label_path)

    horz_img.close()


  ############################################################################
  # Rotate the original image 90 degrees clockwise
  if rotate90clockwise:

    # rotate the image
    rotate90clockwise_img = original_img.transpose(method=Image.ROTATE_270)

    # save the image in the new path
    image_format = image.split('.')[-1]
    image_name = image.split('/')[-1]
    new_image_path = output_folder_path + 'images/' + image_name.replace('.' + image_format , '_rotated90_clockwise.' + image_format)
    rotate90clockwise_img.save(new_image_path)

    # modify the labels
    label_file_format = labels_list.split('.')[-1]
    label_file_name = labels_list.split('/')[-1]
    new_label_path = output_folder_path + 'labels/' + label_file_name.replace('.' + label_file_format , '_rotated90_clockwise.' + label_file_format)
    Label_Modify_Rotate90_Clockwise_Image(labels_list , new_label_path , height , width)

    rotate90clockwise_img.close()


  ############################################################################
  # Rotate the original image 90 degrees counter-clockwise
  if rotate90counterclockwise:

    # rotate the image
    rotate90counter_img = original_img.transpose(method=Image.ROTATE_90)

    # save the image in the new path
    image_format = image.split('.')[-1]
    image_name = image.split('/')[-1]
    new_image_path = output_folder_path + 'images/' + image_name.replace('.' + image_format , '_rotated90_counter_clockwise.' + image_format)
    rotate90counter_img.save(new_image_path)

    # modify the labels
    label_file_format = labels_list.split('.')[-1]
    label_file_name = labels_list.split('/')[-1]
    new_label_path = output_folder_path + 'labels/' + label_file_name.replace('.' + label_file_format , '_rotated90_counter_clockwise.' + label_file_format)
    Label_Modify_Rotate90_ConterClockwise_Image(labels_list , new_label_path , height , width)

    rotate90counter_img.close()
  
  original_img.close()

In [ ]:
def Select_x_Random_Images(total_number_of_images: int ,how_many_images_to_select: int):
  return random.sample(range(0 , total_number_of_images) , how_many_images_to_select)

In [ ]:
def Apply_Augmentations(input_path: str , output_augmentations_path: str , percentage: float , horz_flip: bool , vert_flip: bool , rotate90clockwise: bool , rotate90counterclockwise: bool):
  # "percentage" equals how many of the images we want to augmented [move between none(0) to all(1)]

  # create the output folders
  if not os.path.exists(output_augmentations_path + 'images/'):
    os.makedirs(output_augmentations_path + 'images/')
  if not os.path.exists(output_augmentations_path + 'labels/'):
    os.makedirs(output_augmentations_path + 'labels/')

  # list of all the images in the folder
  image_names = glob.glob(input_path + 'images/*.jpg') 

  # create a dictionary with the different types of augmentations as the keys and if to use them or not (True or False) as values
  augmentations_types_dict = {'horz_flip' : horz_flip , 'vert_flip' : vert_flip , 'rotate90clockwise' : rotate90clockwise , 'rotate90counterclockwise' : rotate90counterclockwise}

  # calculate how many types of augmentations we want to use
  number_of_augmentations_types = 0
  for key in augmentations_types_dict.keys():
    if augmentations_types_dict[key]: # if value is True
      number_of_augmentations_types += 1

  augmentations_types_dict_indexes = {}
  # for each key assign as value a list (can be empty) of indexes. Each index is the image needed to be augmented with the augmentation "key".
  for key in augmentations_types_dict.keys():
    if augmentations_types_dict[key]: # if value is True
      images_to_augment = Select_x_Random_Images(len(image_names) , int((len(image_names) * percentage) // number_of_augmentations_types)) # (total number of images to augment) / (number of augmentations types to use) = 
                                                                                                                                                                # (how many images to augment of each type)
      augmentations_types_dict_indexes[key] = images_to_augment
    else: # if value is False
      augmentations_types_dict_indexes[key] = []

  # reset the augmentations types dictionary so we can modify according to the indexes
  for key in augmentations_types_dict.keys():
    augmentations_types_dict[key] = False

  i = 0
  for imname in image_names:
    image_file_name = imname.split('/')[-1] # find the image actual name
    label_file_name = image_file_name.replace('.jpg' , '.txt') # change the format of the file to the labels file format
    labname = input_path + 'labels/' + label_file_name # the corresponding label path to the current image

    for key in augmentations_types_dict_indexes.keys():
      if len(augmentations_types_dict_indexes[key]) > 0: # if the list of indexes is not empty
        if i in augmentations_types_dict_indexes[key]: # if the current image need to be augmented
          augmentations_types_dict[key] = True

      Image_Augmentations(imname , labname, output_augmentations_path , 
                          horz_flip = augmentations_types_dict['horz_flip'] , 
                          vert_flip = augmentations_types_dict['vert_flip'] , 
                          rotate90clockwise = augmentations_types_dict['rotate90clockwise'] , 
                          rotate90counterclockwise = augmentations_types_dict['rotate90counterclockwise'])
      
      augmentations_types_dict[key] = False
    
    i += 1


# ***Augmantions:***

In [ ]:
# Choose the desired classes
class_list = ['cupmark' , 'cistern']

input_path = dataset.location
for count , class_name in enumerate(class_list):
  if count == 0:
    path_string = '_' + class_name
  else:
    path_string += '_' + class_name

filtered_path = os.path.join(input_path , 'ClassFiltered' + path_string + '/')
if not os.path.exists(filtered_path):
  os.makedirs(filtered_path)

# copy the yaml file of the data to the new agumented folder
shutil.copy2(os.path.join(input_path ,'data.yaml') , filtered_path)

# read yaml file
with open(os.path.join(filtered_path ,'data.yaml')) as file:
  data_file = yaml.safe_load(file)
file.close()

# find the old indexes of the relevant classes
old_class_list_indexes = np.zeros(len(class_list) , dtype=np.uint8)
all_classes = data_file['names']
for i , name in enumerate(class_list):
  for j , name2 in enumerate(all_classes):
    if name2 == name:
      old_class_list_indexes[i] = j

# create a new list of indexes of the relevant classes
new_class_list_indexes = np.arange(len(class_list), dtype=int)

# change yaml file
for key in data_file.keys():
  if key == 'train':
    data_file[key] = os.path.join(filtered_path , 'train/images')

  if key == 'val':
    data_file[key] = os.path.join(filtered_path , 'valid/images')

  if key == 'nc':
    data_file[key] = len(class_list)

  if key == 'names':
    data_file[key] = class_list

# write the updates to the yaml file
with open(os.path.join(filtered_path ,'data.yaml'), 'w') as file:
  yaml.dump(data_file, file)
file.close()

if os.path.exists(os.path.join(input_path , 'train')):
  Filter_Classes(os.path.join(input_path , 'train'),
                 os.path.join(filtered_path ,'train'),
                 old_class_list_indexes,
                 new_class_list_indexes)

if os.path.exists(os.path.join(input_path , 'valid')):
  Filter_Classes(os.path.join(input_path , 'valid'),
                 os.path.join(filtered_path ,'valid'),
                 old_class_list_indexes,
                 new_class_list_indexes)

if os.path.exists(os.path.join(input_path , 'test')):
  Filter_Classes(os.path.join(input_path , 'test'),
                 os.path.join(filtered_path ,'test'),
                 old_class_list_indexes,
                 new_class_list_indexes)

In [ ]:
input_path = filtered_path
grayscale_path = input_path + 'grayscale/'

if os.path.exists(input_path + 'train/'):
  Convert_Grayscale(input_path + 'train/' , grayscale_path + 'train/')

if os.path.exists(input_path + 'valid/'):
  Convert_Grayscale(input_path + 'valid/' , grayscale_path + 'valid/')

if os.path.exists(input_path + 'test/'):
  Convert_Grayscale(input_path + 'test/' , grayscale_path + 'test/')

# copy the yaml file of the data to the new agumented folder
shutil.copy2(input_path + 'data.yaml' , grayscale_path)

# update the train and val paths in the yaml file:
  # read
with open(grayscale_path + 'data.yaml') as file:
    data_file = yaml.safe_load(file)
file.close()

  # change
for key in data_file.keys():
  if key == 'train':
    data_file[key] = grayscale_path + 'train/images'

  if key == 'val':
    data_file[key] = grayscale_path + 'valid/images'

  # wrtie
with open(grayscale_path + 'data.yaml', 'w') as file:
  yaml.dump(data_file, file)
file.close()

In [ ]:
input_path = grayscale_path
augmentations_path = input_path + 'augmented/'
image_precentage_to_augment = 0.7 # (between 0 to 1)

if os.path.exists(input_path + 'train/'):
  Apply_Augmentations(input_path + 'train/' , augmentations_path + 'train/' , image_precentage_to_augment ,
                      horz_flip=True , vert_flip=True , rotate90clockwise=True , rotate90counterclockwise=True)

if os.path.exists(input_path + 'valid/'):
  Apply_Augmentations(input_path + 'valid/' , augmentations_path + 'valid/' , image_precentage_to_augment ,
                      horz_flip=True , vert_flip=True , rotate90clockwise=True , rotate90counterclockwise=True)

if os.path.exists(input_path + 'test/'):
  Apply_Augmentations(input_path + 'test/' , augmentations_path + 'test/' , image_precentage_to_augment ,
                      horz_flip=True , vert_flip=True , rotate90clockwise=True , rotate90counterclockwise=True)

# copy the yaml file of the data to the new agumented folder
shutil.copy2(input_path + 'data.yaml' , augmentations_path)

# update the train and val paths in the yaml file:
  # read
with open(augmentations_path + 'data.yaml') as file:
    data_file = yaml.safe_load(file)
file.close()

  # change
for key in data_file.keys():
  if key == 'train':
    data_file[key] = augmentations_path + 'train/images'

  if key == 'val':
    data_file[key] = augmentations_path + 'valid/images'

  # wrtie
with open(augmentations_path + 'data.yaml', 'w') as file:
  yaml.dump(data_file, file)
file.close()

# ***Utils for Sliding Window:***

In [ ]:
# generate tuple of 1. (x,y) the upper left coordinate and 2. the window itself
def sliding_window(image: np.ndarray, stepSize: int, windowSize: tuple):  # window size: width x height
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # generate a window only if there is still space
            if ((x + windowSize[0]) <= image.shape[1]) and ((y + windowSize[1]) <= image.shape[0]):
                # yield the current window
                yield x, y, Image.fromarray(
                    image[y:y + windowSize[1], x:x + windowSize[0]].astype(np.uint8))  # return the window as PIL image

In [ ]:
def Label_Modify_Sliding_Window(labels_list, output_label_path: str, output_image_path: str,
                                upper_left_corner: tuple, window):
    """ 1. upper_left_corner is in the format of (x , y)
        2. window_size and image_size are in the format of (width , height)
        3. window and label_list are already open   """

    # get the window's size
    window_size = window.size

    # The window's edges (its columns and rows) relative to the original image
    window_upperEdge = upper_left_corner[1]
    window_bottomEdge = upper_left_corner[1] + window_size[1]
    window_leftEdge = upper_left_corner[0]
    window_rightEdge = upper_left_corner[0] + window_size[0]

    window_labels = []
    for row in labels_list.iterrows():  # iterating over each label in the file (we remember that the sizes are
        # normalized)

        # The bounding box edges (their columns and rows) relative to the original image
        label_upperEdge = row[1]['y1'] - 0.5 * row[1]['h']
        label_bottomEdge = row[1]['y1'] + 0.5 * row[1]['h']
        label_leftEdge = row[1]['x1'] - 0.5 * row[1]['w']
        label_rightEdge = row[1]['x1'] + 0.5 * row[1]['w']

        if (label_upperEdge >= window_upperEdge) and (label_bottomEdge <= window_bottomEdge) and (
                label_leftEdge >= window_leftEdge) and (
                label_rightEdge <= window_rightEdge):  # if this True then the entire bounding box is inside the window

            # The bounding box edges (their columns and rows) relative to the window (meaning that now (x =
            # window_leftEdge , y = window_upperEdge) is the point (0,0))
            label_upperEdge = label_upperEdge - window_upperEdge
            label_leftEdge = label_leftEdge - window_leftEdge

            # calculate new coordinate for the label (relative to the window) and normalize them with the window size
            new_x1 = (label_leftEdge + 0.5 * row[1]['w']) / window_size[0]
            new_y1 = (label_upperEdge + 0.5 * row[1]['h']) / window_size[1]
            new_w = row[1]['w'] / window_size[0]
            new_h = row[1]['h'] / window_size[1]

            window_labels.append([int(row[1]['class']), new_x1, new_y1, new_w, new_h])

    flag_window_saved = 0
    if len(window_labels) > 0:  # if there are any labels for this window, save them and the window
        window_df = pd.DataFrame(window_labels,
                                 columns=['class', 'x1', 'y1', 'w', 'h'])  # create a data frame from the new labels
        window_df.to_csv(output_label_path, sep=' ', index=False, header=False,
                         float_format='%.6f')  # save the new data frame as the new label file
        window.save(output_image_path)  # save the window
        flag_window_saved = 1

    return flag_window_saved

In [ ]:
def Random_Tiles_Selector(total_number_of_tiles: int , threshold: int , tiles_folder_path: str):
  
  """
  Our images are too big thus tons of tiles are created.
  This function make sure to modulate the number of tiles we have by randomly remove tiles so we will only have threshold number of tiles.
  """

  if total_number_of_tiles > threshold:
    print(f'\nNumber of total tiles passed {threshold} - Starting to remove tiles randomly:')
    sleep(5)
    selected_image_index = random.sample(range(0 , total_number_of_tiles) , total_number_of_tiles - threshold) # list of indexes of images we want to remove
    image_names = glob.glob(tiles_folder_path + 'images/*.jpg') # list of all the images paths

    for i in selected_image_index:
      imname = image_names[i]

      image_file_name = imname.split('/')[-1]  # find the image actual name
      image_format = imname.split('.')[-1]  # find the image format
      label_file_name = image_file_name.replace('.' + image_format,
                                                '.txt')  # change the format of the file to the labels file format
      labname = tiles_folder_path + 'labels/' + label_file_name  # the corresponding label path to the current image

      os.remove(imname)
      print('\t' + imname + 'was removed from images folder')
      os.remove(labname)
      print('\t' + labname + 'was removed from labels folder\n')
    
    total_number_of_tiles = threshold

  return total_number_of_tiles

In [ ]:
def Apply_Sliding_Window(input_path: str, output_tiles_path: str, step_size: int, window_size: tuple , threshold: int):
    # window_size = (Width x Height)

    # create the output folders
    if not os.path.exists(output_tiles_path + 'images/'):
        os.makedirs(output_tiles_path + 'images/')
        print('\t A new folder was created: ./tiles/images/')
        sleep(2) 
    if not os.path.exists(output_tiles_path + 'labels/'):
        os.makedirs(output_tiles_path + 'labels/')
        print('\t A new folder was created: ./tiles/labels/')
        sleep(10) 

    total_number_of_tiles = 0

    image_names = glob.glob(input_path + 'images/*.jpg')
    for imname in image_names:  # run over all the images
        image_file_name = imname.split('/')[-1]  # find the image actual name
        image_format = imname.split('.')[-1]  # find the image format
        label_file_name = image_file_name.replace('.' + image_format,
                                                  '.txt')  # change the format of the file to the labels file format
        labname = input_path + 'labels/' + label_file_name  # the corresponding label path to the current image

        image = Image.open(imname)  # open image
        labels_list = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])  # read labels

        # scale the bounding box to match original image size
        labels_list[['x1', 'w']] = labels_list[['x1', 'w']] * image.size[0]
        labels_list[['y1', 'h']] = labels_list[['y1', 'h']] * image.size[1]

        print('\n')
        print('\t Open image: ' + image_file_name)
        sleep(10) 

        i = 0  # number of tiles for each image
        for (x, y, window) in sliding_window(np.array(image, dtype=np.uint8), stepSize=step_size,
                                             windowSize=window_size):  # for each image, run over all the created
            # windows

            print(f'\t\t Window ({x} , {y})')

            # create the image's new path
            new_image_path = output_tiles_path + 'images/' + image_file_name.replace('.' + image_format,
                                                                                     f'_tile_{i}.' + image_format)

            # modify the labels
            new_label_path = output_tiles_path + 'labels/' + label_file_name.replace('.txt', f'_tile_{i}.txt')
            flag_window_saved = Label_Modify_Sliding_Window(labels_list, new_label_path, new_image_path,
                                                            (x, y), window)

            i += flag_window_saved
            window.close()

        total_number_of_tiles += i

        total_number_of_tiles = Random_Tiles_Selector(total_number_of_tiles, threshold , output_tiles_path)

        image.close()
        del labels_list
        gc.collect()

# ***Sliding Windows:***

In [ ]:
input_path = augmentations_path           # './datasets/downloaded_dataset/augmented/
tiles_path = input_path + 'tiles/'        # './datasets/downloaded_dataset/augmented/tiles/

step_size = 64
window_size = (256,256)
max_number_of_images = 10000

if os.path.exists(input_path + 'train/'):
  print('Beginning to work on training data:\n')
  sleep(10) 
  Apply_Sliding_Window(input_path + 'train/' , tiles_path + 'train/' , step_size , window_size, int(0.7 * max_number_of_images))
  print('\n Training data work finished!\n')
  sleep(10) 

if os.path.exists(input_path + 'valid/'):
  print('Beginning to work on validation data:\n')
  sleep(10) 
  Apply_Sliding_Window(input_path + 'valid/' , tiles_path + 'valid/' , step_size, window_size , int(0.2 * max_number_of_images))
  print('\n Validation data work finished!\n')
  sleep(10) 

if os.path.exists(input_path + 'test/'):
  print('Beginning to work on test data:\n')
  sleep(10) 
  Apply_Sliding_Window(input_path + 'test/' , tiles_path + 'test/' , step_size, window_size , int(0.1 * max_number_of_images))
  print('\n Test data work finished!\n')
  sleep(10) 

# copy the yaml file of the data to the new agumented folder
shutil.copy2(input_path + 'data.yaml' , tiles_path)

# update the train and val paths in the yaml file:
  # read
with open(tiles_path + 'data.yaml') as file:
    data_file = yaml.safe_load(file)
file.close()

  # change
data_file = Update_Classes(tiles_path, data_file)

  # wrtie
with open(tiles_path + 'data.yaml', 'w') as file:
  yaml.dump(data_file, file)
file.close()

Streaming output truncated to the last 5000 lines.
		 Window (1216 , 1088)
		 Window (1280 , 1088)
		 Window (1344 , 1088)
		 Window (1408 , 1088)
		 Window (1472 , 1088)
		 Window (1536 , 1088)
		 Window (1600 , 1088)
		 Window (1664 , 1088)
		 Window (1728 , 1088)
		 Window (1792 , 1088)
		 Window (1856 , 1088)
		 Window (1920 , 1088)
		 Window (1984 , 1088)
		 Window (2048 , 1088)
		 Window (2112 , 1088)
		 Window (2176 , 1088)
		 Window (2240 , 1088)
		 Window (2304 , 1088)
		 Window (2368 , 1088)
		 Window (2432 , 1088)
		 Window (2496 , 1088)
		 Window (2560 , 1088)
		 Window (2624 , 1088)
		 Window (2688 , 1088)
		 Window (2752 , 1088)
		 Window (2816 , 1088)
		 Window (2880 , 1088)
		 Window (2944 , 1088)
		 Window (3008 , 1088)
		 Window (3072 , 1088)
		 Window (3136 , 1088)
		 Window (3200 , 1088)
		 Window (3264 , 1088)
		 Window (3328 , 1088)
		 Window (3392 , 1088)
		 Window (3456 , 1088)
		 Window (3520 , 1088)
		 Window (3584 , 1088)
		 Window (3648 , 1088)
		 Window (37

**Note:
Possible Improvment:**
We can create even more tiles if we will do a second iteration of the window, but this time from right to left.
If the window + step size don't cover the image perfectly and there are leftovers near the right edge of the image, an iteration from right to left will results in different tiles and will cover the leftovers from the previous iteration.

In [ ]:
# call sliding_window on every image
# for each image divide the labels to the correct tiles (inculuding adjusment of the boundery box's size to the tile)
# tiles without labels are discarded

# What we end up with?
  # dataset_name/augmented/tiles/ :
  #                  train/ :
  #                       images/
  #                       labels/
  #                  valid/ :
  #                       images/
  #                       labels/
  #                  test/  :
  #                       images/
  #                       labels/
  #                  dataset.yaml (updated with the currect train and valid paths)

# **Display Samples:**

In [ ]:
paths_list = []

# Display the images (5 random images from train/valid/test each)
if os.path.exists(tiles_path + 'train/'):
  paths_list.append(tiles_path + 'train/')

if os.path.exists(tiles_path + 'valid/'):
  paths_list.append(tiles_path + 'valid/')

if os.path.exists(tiles_path + 'test/'):
  paths_list.append(tiles_path + 'test/')

# Create figure and axes
fig,ax = plt.subplots(5, len(paths_list), figsize=(45,45))

for col , path in enumerate(paths_list):
  image_names = glob.glob(path + 'images/*.jpg')
  selected_images_indexes = Select_x_Random_Images(len(image_names) , 5)
  selected_images = []
  for i in selected_images_indexes:
    selected_images.append(image_names[i])

  for row , imname in enumerate(selected_images):  # run over all the images
    image_file_name = imname.split('/')[-1]  # find the image actual name
    image_format = imname.split('.')[-1]  # find the image format
    label_file_name = image_file_name.replace('.' + image_format,
                                              '.txt')  # change the format of the file to the labels file format
    labname = path + 'labels/' + label_file_name  # the corresponding label path to the current image

    image = Image.open(imname)  # open image
    labels_list = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])  # read labels

    # scale the bounding box to match original image size
    labels_list[['x1', 'w']] = labels_list[['x1', 'w']] * image.size[0]
    labels_list[['y1', 'h']] = labels_list[['y1', 'h']] * image.size[1]

    # convert image from PIL to numpy.array
    imr = np.array(image, dtype=np.uint8)

    # Display the image
    ax[row][col].imshow(imr)
    ax[row][col].axis('off')

    # Create a Rectangle patch
    for box in labels_list.iterrows():  # iterating over each label in the file
      rect = patches.Rectangle((box[1]['x1']-(box[1]['w']/2) , box[1]['y1']-(box[1]['h']/2)) , 
                               box[1]['w'] , box[1]['h'] , linewidth=2 , edgecolor='g' , facecolor='none')
      # Add the patch to the axes
      ax[row][col].add_patch(rect)
    
    image.close()
    del labels_list
    gc.collect()

plt.show()

# ***YOLOv5 Model:***

In [ ]:
# change working directory and verify files are accessible
path = tiles_path
%cd {path}
%ls -l

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd {path}'/yolov5'
%pip install -r requirements.txt

%pip install "wandb==0.12.10"

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# ***Train:***

In [ ]:
!python train.py \
  --weights yolov5s.pt \
  --img 256 \
  --data {path}/data.yaml \
  --epochs 1000 \
  --batch-size -1 \
  --cache \
  --upload_dataset \
  --bbox_interval 1 \
  --save-period 1

# ***Resume Training (if needed):***

In [ ]:
path = '/content/drive/MyDrive/Colab_Notebooks/tiles/grayscale'

%cd {path}'/yolov5'
%pip install -r requirements.txt
%pip install "wandb==0.12.10"

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# ***Detect:***

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 256 --conf 0.25 --source {path}/test/images